In [ ]:
experiment = 'iw_loss.yaml'
%run -i ../template.ipynb

In [ ]:
body

In [ ]:
def list_of_runs(input_list):
    runs = []
    for k in input_list:
        run = k.split('_')[1]
        if run not in runs:
            runs.append(run)
    return runs 

In [ ]:
runs = os.listdir(netsdir)

runs = [x for x in runs if prefix in x]
loruns = list_of_runs(runs)  # to iterate over

print(len(loruns))

In [ ]:
info

In [ ]:
df = pd.DataFrame()

for r in tqdm(range(len(loruns))):
    name = loruns[r]
    wildcard = prefix + '_' + name + '*'
    
    path = netsdir
    #dist_dicts = io.load_wildcard(path, wildcard + '_dist')
    info_dicts = io.load_wildcard(path, wildcard + '_info')
    
    info = last(info_dicts)
    #if int(nth(info_dicts, 3)['datetime'][11:13]) < 20:
    #    continue
        
    #loss_dicts = io.load_wildcard(path, wildcard + '_loss')
    #net_dicts = io.load_wildcard(path, wildcard + '_net')
    posterior_dicts = io.load_wildcard(path, wildcard + '_posterior')
    sim_dicts = io.load_wildcard(path, wildcard + '_sim')
    

        
    sim = last(sim_dicts)
    approx_posterior = last(posterior_dicts)

    try:
        posterior_mua = approx_posterior.xs[0].m[0]  # 1
        posterior_cova = approx_posterior.xs[0].S[0,0]  # 1
        posterior_mub = approx_posterior.xs[1].m[0]  # 1
        posterior_covb = approx_posterior.xs[1].S[0,0]  # 1
    except:
        print('nf')
        continue
    
    if posterior_covb > posterior_cova:
        posterior_mu = posterior_mub
        posterior_cov = posterior_covb
        posterior_mu2 = posterior_mua
        posterior_cov2 = posterior_cova
    else:
        posterior_mu = posterior_mua
        posterior_cov = posterior_cova
        posterior_mu2 = posterior_mub
        posterior_cov2 = posterior_covb
    
    true_mu = sim.posterior.xs[0].m[0]  # wide
    true_cov = sim.posterior.xs[0].S[0,0]  # wide
    true_mu2 = sim.posterior.xs[1].m[0]  # wide
    true_cov2 = sim.posterior.xs[1].S[0,0]  # wide
    

    iw_loss = info['iw_loss']
    svi = info['svi']
    prior_alpha = info['prior_alpha']
    prior_true = type(nth(info_dicts, -1)['prior_true'])
    prior_true = 'Gaussian' in str(prior_true)

    dim = nth(sim_dicts, -1).dim

    #n_samples = len(last(dist_dicts))

    seed = first(info_dicts)['seed']


    if info['loss_calib'] is None:
        lc = 0.
    else:
        lc = 1.

    data = [
                [name],
                [seed],
                [dim],
                [iw_loss],
                [svi],
                [prior_alpha],
                [prior_true],
                [posterior_mu],
                [posterior_mu2],
                [posterior_cov],
                [posterior_cov2],
                [true_mu],
                [true_cov],
                [true_mu2],
                [true_cov2],
                [info['reg_scale_iw']],
                [info['keep_n']],
                [info['ess']],
                [info['ess_lc']],
                [lc],
                [info['normalize_weights']]
            ]

    data = pd.DataFrame(data).T
    data.columns = ['name', 'seed', 'dim', 'iw_loss', 'svi', 'prior_alpha', 'prior_true', 'posterior_mu', 'posterior_mu2', 
                    'posterior_cov', 'posterior_cov2', 'true_mu','true_cov','true_mu2', 'true_cov2','rsiw','keep','ess','ess_lc','loss_calib', 'norm']

    df = df.append(data)    
        
#df['abserr_cov_tp'] = np.abs(df['t_cov'] - df['p_cov'])
#df['abserr_mu_tp'] = np.abs(df['t_mu'] - df['p_mu'])

df = df.reset_index()
df = df.drop('index', 1)

print(len(df))
df.head()

df.to_pickle('iw_loss.pkl')


In [ ]:
info

In [ ]:
np.sum(df['loss_calib'])

In [ ]:
#dfa = df.copy()
dfc = df.copy()

In [ ]:
subset = dfc.query('dim == 1 & iw_loss ==  True & svi == False').copy()  #  & prior_true == False & keep == True').copy()
subset = subset.reset_index()
subset = subset.drop('index', 1)

sns.set(font_scale=2)
g = sns.factorplot(x="prior_alpha", y="posterior_cov2", hue="norm", data=subset,
                   capsize=.1, palette="YlGnBu_d", size=10, aspect=1., markers='.',
                   linestyles='', legend=True, row="keep", col="ess", sharey=True)
ax = plt.gca()
ax.set_ylim([0, 0.02])
#ax.set_ylim([0, 0.05])
#ax.semilogy()
#saverep('loss_calib')

In [ ]:
subset = dfc.query('dim == 1 & iw_loss == False & svi == True').copy()
subset = subset.reset_index()
subset = subset.drop('index', 1)

plt.plot(subset['posterior_cov2'])

ax.set_ylim([0, 0.05])

#ax.set_ylim([0, 0.01])
#ax.semilogy()
#saverep('loss_calib')

In [ ]:
subset['']

In [ ]:
subset

In [ ]:
subset = dfc.query('dim == 1 & iw_loss == False & svi == False').copy()
subset = subset.reset_index()
subset = subset.drop('index', 1)

plt.hist(subset['abserr_cov_tp'])

In [ ]:
subset = dfc.query('dim == 1 & iw_loss == False & svi == True').copy()
subset = subset.reset_index()
subset = subset.drop('index', 1)

plt.hist(subset['abserr_cov_tp'])